<a href="https://colab.research.google.com/github/valentinaanthonio/single_cell/blob/main/single_cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%cd /content/drive/MyDrive/kaggle competition

/content/drive/MyDrive/kaggle competition


In [10]:
!ls

 cells	'project outline.gdoc'	 scikit-optimize   Untitled0.ipynb


In [4]:
%cd scikit-optimize
!pip install -r requirements.txt
!python setup.py develop


/content/drive/MyDrive/kaggle competition/scikit-optimize
Partial import of skopt during the build process.
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` dir

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import make_scorer
from time import time
import pprint
import joblib
from functools import partial
# Suppressing warnings because of skopt verbosity
import warnings
warnings.filterwarnings("ignore")
# Classifiers
import lightgbm as lgb
# Model selection
from sklearn.model_selection import KFold
# Metrics
from sklearn.metrics import mean_squared_error
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Categorical, Integer
from sklearn.tree import ExtraTreeRegressor
from scipy.stats import mode
from sklearn.model_selection import RandomizedSearchCV

In [6]:
#!unzip -u "open-problems-single-cell-perturbations.zip" -d "cells"

In [11]:
data = pd.read_parquet("cells/de_train.parquet")
data.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [12]:
sample_df = pd.read_csv("cells/sample_submission.csv")
sample_df.head()

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
id_maps = pd.read_csv("cells/id_map.csv")
id_maps.head()

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867


In [14]:
id_maps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         255 non-null    int64 
 1   cell_type  255 non-null    object
 2   sm_name    255 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.1+ KB


In [15]:
data.sum().isnull()/data.shape[0]

cell_type      0.0
sm_name        0.0
sm_lincs_id    0.0
SMILES         0.0
control        0.0
              ... 
ZXDB           0.0
ZXDC           0.0
ZYG11B         0.0
ZYX            0.0
ZZEF1          0.0
Length: 18216, dtype: float64

In [16]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
A1BG,614.0,0.366769,1.634697,-3.258689,-0.215161,0.082722,0.528015,22.085428
A1BG-AS1,614.0,0.250951,1.188660,-4.067518,-0.267728,0.102076,0.523162,14.421991
A2M,614.0,0.148259,2.275410,-28.552191,-0.230129,0.044717,0.405858,27.557166
A2M-AS1,614.0,0.155733,2.184095,-17.801989,-0.229460,0.036156,0.532216,27.603715
A2MP1,614.0,0.684275,3.189206,-6.378904,-0.190989,0.116411,0.616725,35.278040
...,...,...,...,...,...,...,...,...
ZXDB,614.0,0.382672,1.430553,-2.617712,-0.135779,0.145279,0.600652,23.089832
ZXDC,614.0,0.240726,0.918039,-8.310726,-0.141982,0.137559,0.510566,10.844942
ZYG11B,614.0,0.161087,0.779918,-3.456731,-0.169364,0.062804,0.487592,4.822008
ZYX,614.0,-0.160735,0.932961,-7.189487,-0.479099,-0.083260,0.266170,7.826692


In [17]:
data.groupby(['cell_type','sm_name']).sum().T

cell_type   B cells                                                         \
sm_name   Alvocidib Belinostat CHIR-99021 Crizotinib Dabrafenib Dactolisib   
control    0.000000   1.000000   0.000000   0.000000   1.000000   0.000000   
A1BG       5.973634   2.225073  -3.258689  -0.221234   0.942197  -0.505758   
A1BG-AS1   1.339989   0.564347  -0.175129  -0.588647   0.539713   1.495231   
A2M        7.261231  -0.437639   0.012211   0.021279  -0.417087  -0.044533   
A2M-AS1    8.021802  -0.415907  -1.396070   1.104213   0.391397   0.420406   
...             ...        ...        ...        ...        ...        ...   
ZXDB       2.895881  -1.788311  -0.371927   0.603842   0.277953   0.253796   
ZXDC       2.974078  -0.418805  -0.476411   1.152725   1.126481   0.079594   
ZYG11B    -0.076024   3.396236   0.108601   0.599055   0.432023   0.319730   
ZYX        1.453228  -0.567121   0.180639   0.253165  -2.508760  -0.794361   
ZZEF1     -0.159255   0.501423  -0.061943   0.762857  -0.354758   0.799401   

cell_type                                              ... T regulatory cells  \
sm_name   Foretinib Idelalisib LDN 193189 Linagliptin  ...          Tivozanib   
control    0.000000   0.000000   0.000000    0.000000  ...           0.000000   
A1BG      -0.176871   0.394173   0.555620   -0.096688  ...           0.018065   
A1BG-AS1   0.320933  -0.153824  -0.138248    0.057827  ...          -1.640901   
A2M        0.532543   0.178232  -0.153085    0.261704  ...          -0.538470   
A2M-AS1    0.570455   0.566241  -0.325607    0.054919  ...          -0.972873   
...             ...        ...        ...         ...  ...                ...   
ZXDB       0.858617   1.101790  -0.675370   -0.091224  ...          -0.065740   
ZXDC       0.271281   0.061803   0.379266   -0.721902  ...          -0.034910   
ZYG11B    -0.311170   1.406335   0.121937   -0.098806  ...           0.025146   
ZYX       -0.230995  -0.264996  -0.920557   -1.570230  ...           0.071188   
ZZEF1     -1.423259  -0.119743  -0.249233   -0.013876  ...          -1.079043   

cell_type                                                             \
sm_name   Topotecan Tosedostat Trametinib UNII-BXU45ZH6LI Vandetanib   
control    0.000000   0.000000   0.000000        0.000000   0.000000   
A1BG      -0.187309   0.420173   1.157050        0.908787  -0.077971   
A1BG-AS1  -0.618707   0.541773   0.127642        0.731311  -1.192314   
A2M       -0.915560   0.668488   0.583566        1.374410  -0.569594   
A2M-AS1   -0.142321   0.387446   1.793973        1.055129  -0.913077   
...             ...        ...        ...             ...        ...   
ZXDB      -0.234551   1.300374   0.284609        1.184217   0.338837   
ZXDC       0.896681   0.171010   0.339929        0.012347  -0.070177   
ZYG11B    -0.512739   1.451861   0.580874       -0.152790  -0.019641   
ZYX        0.858245   0.225820  -0.571764        0.334035   0.375795   
ZZEF1     -0.957508   0.104604  -0.076428       -0.095420   0.399660   

cell_type                                             
sm_name   Vanoxerine Vardenafil Vorinostat  YK 4-279  
control     0.000000   0.000000   0.000000  0.000000  
A1BG        0.030035  -0.628011   2.423859 -0.156588  
A1BG-AS1   -0.393689   0.676734   1.249268 -0.056493  
A2M         0.586666  -0.412974   2.952193 -0.107307  
A2M-AS1     1.727518  -0.754233   3.572463  0.074802  
...              ...        ...        ...       ...  
ZXDB        0.009014  -0.354516   1.844621 -0.143175  
ZXDC        0.042365  -0.891566   0.990503  0.085479  
ZYG11B      1.108836  -0.253083   0.348120  0.030831  
ZYX        -0.117170   0.643248  -0.582760 -0.426910  
ZZEF1      -0.577300  -0.318537   0.154089 -0.298112  

[18212 rows x 614 columns]

In [18]:
data['cell_type'].unique()

array(['NK cells', 'T cells CD4+', 'T cells CD8+', 'T regulatory cells',
       'B cells', 'Myeloid cells'], dtype=object)

In [19]:
id_maps['cell_type'].unique()

array(['B cells', 'Myeloid cells'], dtype=object)

In [20]:
id_maps = id_maps.drop('id',axis=1)

In [21]:
column_diff = list(set(data['sm_name'])- set(id_maps['sm_name']))
column_diff

['Foretinib',
 'Porcn Inhibitor III',
 'LDN 193189',
 'Dactolisib',
 'Dabrafenib',
 'Penfluridol',
 'Alvocidib',
 'Crizotinib',
 'Palbociclib',
 'Idelalisib',
 'Belinostat',
 'MLN 2238',
 'Linagliptin',
 'O-Demethylated Adapalene',
 'CHIR-99021',
 'R428',
 'Oprozomib (ONX 0912)']

In [24]:
final_data = data.merge(id_maps, how='left', on=['cell_type', 'sm_name'])


In [25]:
final_data['sm_lincs_id'].nunique()

146

In [26]:
final_data['SMILES'].nunique()

146

In [27]:
final_data['control'].value_counts()

False    602
True      12
Name: control, dtype: int64

In [37]:
def preprocessingData(data1, data2,test_split):
  """Takes the train and idmap data and merge them into one on tagret names.
     Control, cell_type and sm_name are encoded with sm_lincs and smiles droped from the dataset
     The function then returns x_train, x_test,y_train,y_test after spliting the data.
  """
  merge_data = data1.merge(data2, how='left', on=['cell_type', 'sm_name'])
  merge_data['control'] = merge_data['control'].map({False: 0, True: 1})
  merge_data = merge_data.drop(['sm_lincs_id','SMILES'], axis=1)
  lb = LabelEncoder()
  merge_data[['cell_type','sm_name']] = (merge_data[['cell_type','sm_name']]
                                         .apply(lambda col : lb.fit_transform(col)
                                         if col.name in ['cell_type','sm_name'] else col))
  features = merge_data.drop(['cell_type','sm_name'],axis=1).astype('float32')
  target = merge_data.loc[:,('cell_type','sm_name')]
  x_train, x_test,y_train,y_test = train_test_split(features, target,test_size=test_split,random_state=12345)
  return x_train, x_test,y_train,y_test

In [ ]:
#final_data['control'] = final_data['control'].map({False: 0, True: 1})

In [29]:
def mrrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred) ** 2, axis=1)))
mrrmse_scorer = make_scorer(mrrmse,greater_is_better=False)

In [ ]:
#final_data = final_data.drop(['sm_lincs_id','SMILES'], axis=1)
#final_data.head()

In [ ]:
#lb = LabelEncoder()
#final_data[['cell_type','sm_name']] = final_data[['cell_type','sm_name']].apply(lambda col : lb.fit_transform(col) if col.name in ['cell_type','sm_name'] else col)

In [ ]:
# features = final_data.drop(['cell_type','sm_name'],axis=1)
# target = final_data.loc[:,('cell_type','sm_name')]

In [ ]:
# features = features.astype('float32')

In [ ]:
# x_train, x_test,y_train,y_test = train_test_split(features, target,test_size=.15,random_state=12345)

In [30]:
def report_perf(optimizer, X, y, title="model"):
    start = time()
    optimizer.fit(X, y)
    d = pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds, candidates checked: %d, best"
           " score: %.5f" % (time() - start, len(optimizer.cv_results_['params']), best_score)))
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [43]:
def model_training(data1, data2, n_splits,test_split):
  """
  Trains an XGBoost regression model with the best hyperparameters found using RandomizedSearchCV.

    Parameters:
        data1 (DataFrame): The first set of data for training and testing.
        data2 (DataFrame): The second set of data for training and testing.
        num_splits (int): Number of splits for K-Fold cross-validation.
        test_split (float): Fraction of data to use for testing.

    Returns:
        None
  """
  # Preprocess the data and split it into training and testing sets
  x_train, x_test,y_train,y_test = preprocessingData(data1, data2,test_split)
  # Define a custom scoring function using mean rowwise root mean squared error
  scoring = make_scorer(partial(mrrmse),greater_is_better=False)
  # Create a K-Fold cross-validator
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)
  # Define the hyperparameter search space
  search_spaces = {'learning_rate': [0.01, 0.1],
                    'num_leaves': range(2, 129),
                    'max_depth': range(-1, 65),
                    'max_bin': range(16, 129),
                    'subsample': [0.01, 0.5],
                    'min_child_weight': [0.01, 3.0],
                    'reg_lambda': [1e-9, 5.0],
                    'reg_alpha': [1e-9, 5.0],
}
  # Create an XGBoost model for hyperparameter optimization
  model = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')
   # Perform hyperparameter optimization using RandomizedSearchCV
  opt = RandomizedSearchCV(estimator=model,
                          param_distributions=search_spaces,
                          scoring=scoring,
                          cv=kf,
                          n_iter=60,
                          n_jobs=-1,
                          return_train_score=False,
                          refit=False,
                          random_state=0
)
  best_params = report_perf(opt,x_train,y_train,'Xgboost')

  # Update XGBoost model with best parameters
  best_model = XGBRegressor(**best_params, n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

  best_model.fit(x_train, y_train)

  # Predict on the test set
  y_pred = best_model.predict(x_test)

  # Evaluate the model's performance on the test set
  mse = mrrmse(y_test, y_pred)
  print(f'Mean Rowwise Root Mean Squared Error on Test Set: {mse}')

  # Optionally, print the best hyperparameters
  print("Best hyperparameters:")
  for param, value in best_params.items():
      print(f"{param}: {value}")


In [44]:
model_training(data1 = data, data2 = id_maps, n_splits=5,test_split=.2)

Xgboost took 1381.73 seconds, candidates checked: 60, best score: -25.13322
Best parameters:
{'learning_rate': 0.1,
 'max_bin': 32,
 'max_depth': 3,
 'min_child_weight': 0.01,
 'num_leaves': 102,
 'reg_alpha': 5.0,
 'reg_lambda': 1e-09,
 'subsample': 0.5}

Mean Rowwise Root Mean Squared Error on Test Set: 25.363128095936723
Best hyperparameters:
subsample: 0.5
reg_lambda: 1e-09
reg_alpha: 5.0
num_leaves: 102
min_child_weight: 0.01
max_depth: 3
max_bin: 32
learning_rate: 0.1


In [45]:
def model_training(data1, data2, n_splits,test_split):
  """
  Trains an XGBoost regression model with the best hyperparameters found using RandomizedSearchCV.

    Parameters:
        data1 (DataFrame): The first set of data for training and testing.
        data2 (DataFrame): The second set of data for training and testing.
        num_splits (int): Number of splits for K-Fold cross-validation.
        test_split (float): Fraction of data to use for testing.

    Returns:
        None
  """
  # Preprocess the data and split it into training and testing sets
  x_train, x_test,y_train,y_test = preprocessingData(data1, data2,test_split)
  # Define a custom scoring function using mean rowwise root mean squared error
  scoring = make_scorer(partial(mrrmse),greater_is_better=False)
  # Create a K-Fold cross-validator
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)
  # Define the hyperparameter search space
  search_spaces = {'learning_rate': [0.01, 0.1],
                  'num_leaves': range(2, 129),
                  'max_depth': range(-1, 65),
                  'max_bin': range(16, 129),
                  'subsample': [0.01, 0.5],
                  'min_child_weight': [0.01, 3.0],
                  'reg_lambda': [1e-9, 5.0],
                  'reg_alpha': [1e-9, 5.0],
                  'gamma': [0.0, 1.0],  # Regularization parameter
                  'colsample_bytree': [0.6, 1.0],  # Fraction of features to be used during training
                  'n_estimators': range(50, 500),  # Number of boosting rounds
                  'scale_pos_weight': [1.0, 10.0],  # Control the balance of positive and negative weights
                  'min_split_loss': [0.0, 0.5],  # Minimum loss reduction required to make a further partition on a leaf node
}
  # Create an XGBoost model for hyperparameter optimization
  model = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')
   # Perform hyperparameter optimization using RandomizedSearchCV
  opt = RandomizedSearchCV(estimator=model,
                          param_distributions=search_spaces,
                          scoring=scoring,
                          cv=kf,
                          n_iter=60,
                          n_jobs=-1,
                          return_train_score=False,
                          refit=False,
                          random_state=0
)
  best_params = report_perf(opt,x_train,y_train,'Xgboost')

  # Update XGBoost model with best parameters
  best_model = XGBRegressor(**best_params, n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

  best_model.fit(x_train, y_train)

  # Predict on the test set
  y_pred = best_model.predict(x_test)

  # Evaluate the model's performance on the test set
  mse = mrrmse(y_test, y_pred)
  print(f'Mean Rowwise Root Mean Squared Error on Test Set: {mse}')

  # Optionally, print the best hyperparameters
  print("Best hyperparameters:")
  for param, value in best_params.items():
      print(f"{param}: {value}")


In [46]:
model_training(data, id_maps, n_splits=5,test_split=.15)

Xgboost took 2816.09 seconds, candidates checked: 60, best score: -24.84090
Best parameters:
{'colsample_bytree': 0.6,
 'gamma': 1.0,
 'learning_rate': 0.01,
 'max_bin': 33,
 'max_depth': 18,
 'min_child_weight': 3.0,
 'min_split_loss': 0.0,
 'n_estimators': 395,
 'num_leaves': 62,
 'reg_alpha': 1e-09,
 'reg_lambda': 5.0,
 'scale_pos_weight': 1.0,
 'subsample': 0.5}

Mean Rowwise Root Mean Squared Error on Test Set: 23.375844613347052
Best hyperparameters:
subsample: 0.5
scale_pos_weight: 1.0
reg_lambda: 5.0
reg_alpha: 1e-09
num_leaves: 62
n_estimators: 395
min_split_loss: 0.0
min_child_weight: 3.0
max_depth: 18
max_bin: 33
learning_rate: 0.01
gamma: 1.0
colsample_bytree: 0.6


In [ ]:
# Create regression models
model_1 = KNeighborsRegressor()
model_2 = RandomForestRegressor(random_state=0)
model_3 = XGBRegressor(random_state=0)

# Preprocess the data and split into training and testing sets
X_train, X_test,y_train, y_test = preprocessingData(data, id_maps, test_split=0.15)

# Train the regression models on the training data
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)

# Make predictions using the trained models
preds = np.stack([model_1.predict(X_test),
                  model_2.predict(X_test),
                  model_3.predict(X_test)]).T

# Calculate the mode (majority voting) of the predictions
min_voting = np.apply_along_axis(mode, 1, preds)[:, 0]

# Calculate discordant predictions
discordant = np.sum(np.var(preds, axis=1) > 0) / len(y_test)
print(f"Discordant Predictions: {discordant:0.2f}")

# Calculate and print mean squared error (MSE) for each individual model
for i, model_name in enumerate(['Knn', 'RF', 'XGB']):
    mse = mrrmse(y_true=y_test, y_pred=preds[:, i])
    print(f"MSE for Model {model_name}: {mse:0.3f}")
# Calculate and print MSE for majority voting
min_voting_score = mrrmse(y_true=y_test, y_pred=min_voting)
print(f"MSE for Majority Voting: {min_voting_score:0.3f}")

In [65]:
scoring = make_scorer(partial(mrrmse),
                                greater_is_better=False)
# Setting the validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=0)
# Setting the basic regressor

# Define the hyperparameter search space
search_spaces = {
    'learning_rate': [0.01, 0.1],
    'num_leaves': range(2, 129),
    'max_depth': range(-1, 65),
    'max_bin': range(16, 129),
    'subsample': [0.01, 0.5],
    'min_child_weight': [0.01, 3.0],
    'reg_lambda': [1e-9, 5.0],
    'reg_alpha': [1e-9, 5.0],
}
xgb = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Create an instance of RandomizedSearchCV
opt = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=search_spaces,
    scoring=scoring,
    cv=kf,
    n_iter=60,
    n_jobs=-1,
    return_train_score=False,
    refit=False,
    random_state=0
)

best_params = report_perf(opt,x_train,y_train,'XGBoost')

XGBoost took 1257.24 seconds, candidates checked: 60, best score: -25.13322
Best parameters:
{'learning_rate': 0.1,
 'max_bin': 32,
 'max_depth': 3,
 'min_child_weight': 0.01,
 'num_leaves': 102,
 'reg_alpha': 5.0,
 'reg_lambda': 1e-09,
 'subsample': 0.5}



In [66]:
xgb = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist',**best_params,metric=mrrmse_scorer)
xgb.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=32,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             metric=make_scorer(mrrmse, greater_is_better=False),
             min_child_weight=0.01, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1, num_leaves=102, ...)

In [67]:
y_pred = xgb.predict(x_test)
score = mrrmse(y_test, y_pred)
print(f'MRRMSE: {score}')

MRRMSE: 25.363128095936723


In [76]:
# Define the hyperparameter search space
search_spaces = {
    'learning_rate': [0.01, 0.1],
    'num_leaves': range(2, 129),
    'max_depth': range(-1, 65),
    'max_bin': range(16, 129),
    'subsample': [0.01, 0.5],
    'min_child_weight': [0.01, 3.0],
    'reg_lambda': [1e-9, 5.0],
    'reg_alpha': [1e-9, 5.0],
}

# Setting the validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Setting the basic regressor
xgb = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Create an instance of RandomizedSearchCV
opt = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=search_spaces,
    scoring=scoring,
    cv=kf,
    n_iter=60,
    n_jobs=-1,
    return_train_score=False,
    refit=False,
    random_state=0
)

# Fit the RandomizedSearchCV
opt.fit(x_train, y_train)

# Get the best parameters
best_params = opt.best_params_

# Create an instance of XGBRegressor with the best parameters
best_model = XGBRegressor(**best_params, n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Train the model on the full training set
best_model.fit(x_train, y_train)

# Get feature importances
feature_importances = best_model.feature_importances_

# Identify the indices of the top k important features
k = 10  # You can adjust this value
top_k_indices = np.argsort(feature_importances)[::-1][:k]

# Get the names of the top k important features
top_k_features = x_train.columns[top_k_indices]

# Print the names of the top k important features
print(f"Top {k} Important Features:")
print(top_k_features)


Top 10 Important Features:
Index(['ZNF548', 'AC118465.1', 'MAML1', 'PMVK', 'DMC1', 'AC245128.3',
       'AC118553.2', 'DMKN', 'RYK', 'ZNF547'],
      dtype='object')


In [77]:
num_features_to_remove = 10
least_important_indices = np.argsort(feature_importances)[:num_features_to_remove]

# Remove the least important features from the training set
x_train_filtered = x_train.drop(x_train.columns[least_important_indices], axis=1)

# Print the columns of the filtered training set
print("Columns after removing the least important features:")
print(x_train_filtered.columns)

Columns after removing the least important features:
Index(['control', 'A1BG', 'A1BG-AS1', 'A2M', 'A2M-AS1', 'A2MP1', 'A4GALT',
       'AAAS', 'AACS', 'AAGAB',
       ...
       'ZUP1', 'ZW10', 'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11B',
       'ZYX', 'ZZEF1'],
      dtype='object', length=18202)


In [79]:
xgb = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist',**best_params,metric=mrrmse_scorer)
xgb.fit(x_train_filtered,y_train)
x_test_filtered = x_test.drop(x_train.columns[least_important_indices], axis=1)
y_pred = xgb.predict(x_test_filtered)
score = mrrmse(y_test, y_pred)
print(f'MRRMSE: {score}')

MRRMSE: 28.06441605700222


In [49]:


# Define your custom scoring function
def mrrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred) ** 2, axis=1)))

# Make it a scoring function
scoring = make_scorer(mrrmse, greater_is_better=False)

# Setting the validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Define the hyperparameter search space
search_spaces = {
    'learning_rate': [0.01, 0.1],
    'num_leaves': range(2, 129),
    'max_depth': range(3, 65),  # Adjusted the starting point for max_depth
    'max_bin': range(16, 129),
    'subsample': [0.1, 0.5],  # Adjusted the starting point for subsample
    'min_child_weight': [0.01, 3.0],
    'reg_lambda': [1e-9, 5.0],
    'reg_alpha': [1e-9, 5.0],
}

# Initialize XGBoost model
xgb = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Create an instance of RandomizedSearchCV
opt = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=search_spaces,
    scoring=scoring,
    cv=kf,
    n_iter=60,
    n_jobs=-1,
    return_train_score=False,
    refit=False,
    random_state=0
)

# Perform hyperparameter tuning
best_params = report_perf(opt, x_train, y_train, 'XGBoost')

# Update XGBoost model with best parameters
best_model = XGBRegressor(**best_params, n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Train the updated model on the training set
best_model.fit(x_train, y_train)

# Predict on the test set
y_pred = best_model.predict(x_test)

# Evaluate the model's performance on the test set
mse = mrrmse(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

# Optionally, print the best hyperparameters
print("Best hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


XGBoost took 591.80 seconds, candidates checked: 60, best score: -26.42993
Best parameters:
{'learning_rate': 0.1,
 'max_bin': 19,
 'max_depth': 28,
 'min_child_weight': 0.01,
 'num_leaves': 121,
 'reg_alpha': 5.0,
 'reg_lambda': 5.0,
 'subsample': 0.1}

Mean Squared Error on Test Set: 25.029312653479593
Best hyperparameters:
subsample: 0.1
reg_lambda: 5.0
reg_alpha: 5.0
num_leaves: 121
min_child_weight: 0.01
max_depth: 28
max_bin: 19
learning_rate: 0.1


In [80]:


# Define your custom scoring function
def mrrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred) ** 2, axis=1)))

# Make it a scoring function
scoring = make_scorer(mrrmse, greater_is_better=False)

# Setting the validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Define the hyperparameter search space
search_spaces = {
    'learning_rate': [0.01, 0.1],
    'num_leaves': range(2, 129),
    'max_depth': range(3, 65),  # Adjusted the starting point for max_depth
    'max_bin': range(16, 129),
    'subsample': [0.1, 0.5],  # Adjusted the starting point for subsample
    'min_child_weight': [0.01, 3.0],
    'reg_lambda': [1e-9, 5.0],
    'reg_alpha': [1e-9, 5.0],
}


# Initialize XGBoost model
xgb = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Create an instance of RandomizedSearchCV
opt = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=search_spaces,
    scoring=scoring,
    cv=kf,
    n_iter=60,
    n_jobs=-1,
    return_train_score=False,
    refit=False,
    random_state=0
)

# Perform hyperparameter tuning
best_params = report_perf(opt, x_train, y_train, 'XGBoost')

# Update XGBoost model with best parameters
best_model = XGBRegressor(**best_params, n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Train the updated model on the training set
best_model.fit(x_train, y_train)

# Predict on the test set
y_pred = best_model.predict(x_test)

# Evaluate the model's performance on the test set
mse = mrrmse(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

# Optionally, print the best hyperparameters
print("Best hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


XGBoost took 1706.52 seconds, candidates checked: 60, best score: -24.84490
Best parameters:
{'learning_rate': 0.1,
 'max_bin': 22,
 'max_depth': 26,
 'min_child_weight': 0.01,
 'num_leaves': 128,
 'reg_alpha': 5.0,
 'reg_lambda': 5.0,
 'subsample': 0.5}

Mean Squared Error on Test Set: 25.24316203343167
Best hyperparameters:
subsample: 0.5
reg_lambda: 5.0
reg_alpha: 5.0
num_leaves: 128
min_child_weight: 0.01
max_depth: 26
max_bin: 22
learning_rate: 0.1


In [55]:


# Define your custom scoring function
def mrrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred) ** 2, axis=1)))

# Make it a scoring function
scoring = make_scorer(mrrmse, greater_is_better=False)

# Setting the validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Define the hyperparameter search space
search_spaces = {
    'max_features': ['auto', 'sqrt', 'log2'],  # The number of features to consider when looking for the best split
    'max_depth': range(10, 50, 5),  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}

# Initialize Extratress model
trees = ExtraTreeRegressor(random_state=0)


# Create an instance of RandomizedSearchCV
opt = RandomizedSearchCV(
    estimator=trees,
    param_distributions=search_spaces,
    scoring=scoring,
    cv=kf,
    n_iter=60,
    n_jobs=-1,
    return_train_score=False,
    refit=False,
    random_state=0
)

# Perform hyperparameter tuning
best_params = report_perf(opt, x_train, y_train, 'ExTraTrees')

# Update XGBoost model with best parameters
best_model = ExtraTreeRegressor(**best_params,  random_state=0)

# Train the updated model on the training set
best_model.fit(x_train, y_train)

# Predict on the test set
y_pred = best_model.predict(x_test)

# Evaluate the model's performance on the test set
mse = mrrmse(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

# Optionally, print the best hyperparameters
print("Best hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


ExTraTrees took 39.21 seconds, candidates checked: 60, best score: -26.87977
Best parameters:
{'max_depth': 10,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 2}

Mean Squared Error on Test Set: 25.901801214433593
Best hyperparameters:
min_samples_split: 2
min_samples_leaf: 1
max_features: log2
max_depth: 10


In [72]:
from functools import partial
from skopt import dummy_minimize, space, gp_minimize
from skopt.utils import use_named_args

In [74]:
scoring = partial(mrrmse)
# Setting the cv strategy
kf = KFold(n_splits=5, shuffle=True, random_state=0)
# Setting the search space
space = [Real(0.01, 1.0, 'uniform', name='learning_rate'),
         Integer(1, 8, name='max_depth'),
         Real(0.1, 1.0, 'uniform', name='subsample'),
         Real(0.1, 1.0, 'uniform', name='colsample_bytree'),  # subsample ratio of columns by tree
         Real(0, 100., 'uniform', name='reg_lambda'),      # L2 regularization
         Real(0, 100., 'uniform', name='reg_alpha'),       # L1 regularization
         Real(1, 30, 'uniform', name='min_child_weight'),     # minimum sum of instance weight (hessian)
]
model = XGBRegressor(n_estimators=10_000,
                     booster='gbtree', random_state=0, tree_method='gpu_hist')

def make_objective(model, X, y_true, space, cv, scoring, validation=0.2):
    @use_named_args(space)
    def objective(learning_rate, max_depth, subsample, colsample_bytree, reg_lambda, reg_alpha, min_child_weight):
        params = {
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree,
            'reg_lambda': reg_lambda,
            'reg_alpha': reg_alpha,
            'min_child_weight': min_child_weight
        }

        model.set_params(**params)

        validation_scores = []

        for train_index, test_index in cv.split(X, y_true):
            train_examples = int(len(train_index) * (1 - validation))
            train_index, val_index = train_index[:train_examples], train_index[train_examples:]

            start_time = time()
            model.fit(X.iloc[train_index, :], y_true.iloc[train_index, :],
                      early_stopping_rounds=50,
                      eval_set=[(X.iloc[val_index, :], y_true.iloc[val_index, :])],
                      verbose=0)
            end_time = time()

            rounds = model.best_iteration
            test_preds = model.predict(X.iloc[test_index, :])
            test_score = scoring(y_true.iloc[test_index, :], test_preds)
            print(f"Validation mrrmse: {test_score:0.5f} - {rounds} rounds - "
                  f"it took {end_time - start_time:0.0f} secs")
            validation_scores.append(test_score)

        return np.mean(validation_scores)

    return objective

def onstep(res):
    global counter
    x0 = res.x_iters   # List of input points
    y0 = res.func_vals # Evaluation of input points
    print('Last eval: ', x0[-1], ' - Score ', y0[-1])
    print('Current iter: ', counter, ' - Best Score ', res.fun, ' - Best Args: ', res.x)
    joblib.dump((x0, y0), 'checkpoint.pkl')  # Saving a checkpoint to disk
    counter += 1

counter = 0
history = {i: list() for i in range(5)}
used_time = 0

objective = make_objective(model,
                           x_train, y_train,
                           space=space,
                           cv=kf,
                           scoring=mrrmse)
gp_round = dummy_minimize(func=objective,
                          dimensions=space,
                          n_calls=30,
                          callback=[onstep],
                          random_state=0)


Validation rmse: 27.60100 - 25 rounds - it took 13 secs
Validation rmse: 26.88491 - 10 rounds - it took 11 secs
Validation rmse: 25.90530 - 2 rounds - it took 10 secs
Validation rmse: 26.63947 - 2 rounds - it took 10 secs
Validation rmse: 25.22890 - 5 rounds - it took 10 secs
Last eval:  [0.5969161720427683, 1, 0.6424870384644795, 0.5903948646972073, 42.36547993389048, 64.58941130666562, 13.690029126618084]  - Score  26.45191497802682
Current iter:  0  - Best Score  26.45191497802682  - Best Args:  [0.5969161720427683, 1, 0.6424870384644795, 0.5903948646972073, 42.36547993389048, 64.58941130666562, 13.690029126618084]
Validation rmse: 25.18399 - 2 rounds - it took 10 secs
Validation rmse: 25.99818 - 2 rounds - it took 11 secs
Validation rmse: 26.04485 - 1 rounds - it took 10 secs
Validation rmse: 25.59769 - 3 rounds - it took 10 secs
Validation rmse: 26.39397 - 1 rounds - it took 11 secs
Last eval:  [0.8928552707742591, 1, 0.3453906651221019, 0.529898605589215, 81.21687287754933, 47.99

In [75]:
# Extract best hyperparameters
best_params = {sp.name: x for sp, x in zip(gp_round.space, gp_round.x)}

# Create an instance of XGBRegressor with the best hyperparameters
best_model = XGBRegressor(**best_params, n_estimators=10_000, booster='gbtree', random_state=0, tree_method='gpu_hist')

# Train the model on your training data
best_model.fit(x_train, y_train, early_stopping_rounds=50, eval_metric='rmse', eval_set=[(x_train, y_train)], verbose=0)

# Make predictions on your test data
y_pred = best_model.predict(x_test)

# Evaluate the performance if needed
mse = mrrmse(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

# Print the best hyperparameters
print(f"Best score: {gp_round.fun:0.5f}")
print("Best hyperparameters:")
for sp, x in zip(gp_round.space, gp_round.x):
    print(f"{sp.name:25} : {x}")


Mean Squared Error on Test Set: 25.386501403587378
Best score: 25.05693
Best hyperparameters:
learning_rate             : 0.26773751831153064
max_depth                 : 6
subsample                 : 0.7334997251863298
colsample_bytree          : 0.19020419858107102
reg_lambda                : 91.94826137446736
reg_alpha                 : 71.42412995491115
min_child_weight          : 29.966563190468133


In [34]:
# Define your custom scoring function
def mrrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred) ** 2, axis=1)))

# Make it a scoring function
scoring = make_scorer(mrrmse, greater_is_better=False)

# Apply log transformation to features
x_train_log = np.log1p(x_train)
x_test_log = np.log1p(x_test)

# Setting the validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Define the hyperparameter search space
search_spaces = {
    'learning_rate': [0.01, 0.1],
    'num_leaves': range(2, 129),
    'max_depth': range(3, 65),  # Adjusted the starting point for max_depth
    'max_bin': range(16, 129),
    'subsample': [0.1, 0.5],  # Adjusted the starting point for subsample
    'min_child_weight': [0.01, 3.0],
    'reg_lambda': [1e-9, 5.0],
    'reg_alpha': [1e-9, 5.0],
}

# Initialize XGBoost model
xgb = XGBRegressor(n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Create an instance of RandomizedSearchCV
opt = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=search_spaces,
    scoring=scoring,
    cv=kf,
    n_iter=60,
    n_jobs=-1,
    return_train_score=False,
    refit=False,
    random_state=0
)

# Perform hyperparameter tuning
best_params = report_perf(opt, x_train_log, y_train, 'XGBoost')

# Update XGBoost model with best parameters
best_model = XGBRegressor(**best_params, n_jobs=-1, verbose=1, random_state=0, tree_method='gpu_hist')

# Train the updated model on the log-transformed training set
best_model.fit(x_train_log, y_train)

# Predict on the log-transformed test set
y_pred_log = best_model.predict(x_test_log)

# Perform inverse log transformation
y_pred = np.expm1(y_pred_log)

# Evaluate the model's performance on the test set
mse = mrrmse(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

# Optionally, print the best hyperparameters
print("Best hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


XGBoost took 2179.00 seconds, candidates checked: 60, best score: -25.55084
Best parameters:
{'learning_rate': 0.1,
 'max_bin': 89,
 'max_depth': 6,
 'min_child_weight': 0.01,
 'num_leaves': 42,
 'reg_alpha': 5.0,
 'reg_lambda': 5.0,
 'subsample': 0.5}

Mean Squared Error on Test Set: inf
Best hyperparameters:
subsample: 0.5
reg_lambda: 5.0
reg_alpha: 5.0
num_leaves: 42
min_child_weight: 0.01
max_depth: 6
max_bin: 89
learning_rate: 0.1


In [40]:
import tensorflow as tf


# Assuming x_train_log and x_test_log are your log-transformed features
# and y_train is your target variable

# Split the data into training and validation sets
features = final_data.drop(['cell_type','sm_name'],axis=1)
target = final_data.loc[:,('cell_type','sm_name')]
x_train, x_test,y_train,y_test = train_test_split(features, target,test_size=.2,random_state=12345)

# Custom MRRMSE loss function
def mrrmse(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)  # Cast to float32
    y_pred = tf.cast(y_pred, dtype=tf.float32)  # Cast to float32

    squared_errors = tf.square(y_true - y_pred)
    mean_squared_errors = tf.reduce_mean(squared_errors, axis=1)
    root_mean_squared_errors = tf.sqrt(mean_squared_errors)
    return tf.reduce_mean(root_mean_squared_errors, axis=0)

# Build a simple neural network model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation='relu', input_dim=x_train.shape[1]))
model.add(tf.keras.layers.Dropout(.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(.5))
model.add(tf.keras.layers.Dense(1, activation='linear'))

# Compile the model with custom loss function
model.compile(optimizer='adam', loss=mrrmse)

# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test), verbose=1)

# Evaluate the model on the validation set
y_pred_nn = model.predict(x_test)

# Evaluate the model's performance on the validation set
mse_nn = mrrmse(y_test, y_pred_nn)
print(f'Mean Squared Error on Validation Set (Neural Network): {mse_nn}')

Epoch 1/50
16/16 [==============================] - 1s 18ms/step - loss: 61.6842 - val_loss: 61.9611
Epoch 2/50
16/16 [==============================] - 0s 7ms/step - loss: 63.2118 - val_loss: 50.1619
Epoch 3/50
16/16 [==============================] - 0s 8ms/step - loss: 55.1678 - val_loss: 50.4446
Epoch 4/50
16/16 [==============================] - 0s 7ms/step - loss: 59.5067 - val_loss: 61.5538
Epoch 5/50
16/16 [==============================] - 0s 7ms/step - loss: 60.1743 - val_loss: 48.2679
Epoch 6/50
16/16 [==============================] - 0s 9ms/step - loss: 53.7265 - val_loss: 46.7474
Epoch 7/50
16/16 [==============================] - 0s 9ms/step - loss: 58.2201 - val_loss: 51.2762
Epoch 8/50
16/16 [==============================] - 0s 8ms/step - loss: 52.7067 - val_loss: 44.0480
Epoch 9/50
16/16 [==============================] - 0s 9ms/step - loss: 52.3292 - val_loss: 45.8971
Epoch 10/50
16/16 [==============================] - 0s 8ms/step - loss: 51.4401 - val_loss: 45.429

In [44]:
from sklearn.preprocessing import StandardScaler

# Normalize the data using StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Define the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(x_train.shape[1], 1), return_sequences=True))
model.add(tf.keras.layers.LSTM(64, return_sequences=True))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dense(1, activation='linear'))

# Compile the model with a custom loss function
def mrrmse(y_true, y_pred):
    # Custom loss function code here
    return tf.reduce_mean(tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred), axis=1)))

model.compile(optimizer='adam', loss=mrrmse)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(x_train.reshape(x_train.shape[0], x_train.shape[1], 1), y_train,
          epochs=50, batch_size=32, validation_data=(x_test.reshape(x_test.shape[0], x_test.shape[1], 1), y_test),
          verbose=1, callbacks=[early_stopping])

# Evaluate the model on the validation set
y_pred_nn = model.predict(x_test.reshape(x_test.shape[0], x_test.shape[1], 1))

# Evaluate the model's performance on the validation set
mse_nn = mrrmse(y_test, y_pred_nn)
print(f'Mean Squared Error on Validation Set (Neural Network with LSTM): {mse_nn}')


Epoch 1/50
16/16 [==============================] - 29s 1s/step - loss: 50.4073 - val_loss: 49.2172
Epoch 2/50
16/16 [==============================] - 22s 1s/step - loss: 47.8699 - val_loss: 47.4047
Epoch 3/50
16/16 [==============================] - 22s 1s/step - loss: 46.5740 - val_loss: 46.5991
Epoch 4/50
16/16 [==============================] - 22s 1s/step - loss: 46.0298 - val_loss: 46.1727
Epoch 5/50
16/16 [==============================] - 22s 1s/step - loss: 45.6305 - val_loss: 45.7704
Epoch 6/50
16/16 [==============================] - 22s 1s/step - loss: 45.3034 - val_loss: 45.4713
Epoch 7/50
16/16 [==============================] - 22s 1s/step - loss: 45.0399 - val_loss: 45.2055
Epoch 8/50
16/16 [==============================] - 22s 1s/step - loss: 44.8068 - val_loss: 44.9694
Epoch 9/50
16/16 [==============================] - 22s 1s/step - loss: 44.5905 - val_loss: 44.7511
Epoch 10/50
16/16 [==============================] - 22s 1s/step - loss: 44.3875 - val_loss: 44.5453

In [ ]:
lr = LinearRegression()
sfs_lr = SFS(lr, verbose=1, n_jobs=-1, scoring=mrrmse_scorer,cv=5,k_features="parsimonious")
sfs_lr.fit(x_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 732 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 1432 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 2332 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 3432 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done 4732 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 6232 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 9832 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 11932 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 14232 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 16732 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 18197 out of 18212 | elapsed:  2.4min remaining:    0.1s
[Parallel(n_jobs=-1)

In [ ]:
pd.DataFrame.from_dict(sfs.get_metric_dict()).T

In [ ]:
X_train_sfs = sfs.transform(x_train)
X_test_sfs = sfs.transform(x_test)

lr.fit(X_train_sfs, y_train)
y_pred = lr.predict(X_test_sfs)

k = len(sfs.k_feature_names_)
n = x_train.shape[0]
score = sfs.k_score_


print(f'k: {k}')
print(list(sfs.k_feature_names_))

In [ ]:

sfs_test = sfs_lr.transform(x_test)
y_pred = sfs_lr.predict(sfs_test)
mrrse_score = mrrmse(y_test,y_pred)
print(f' MRRSE Score: {mrrse_score:.3f}')

In [ ]:
tree = DecisionTreeRegressor(**best_params,random_state=1)
tree.fit(x_train,y_train)
y_pred = tree.predict(x_test)
mrrse_score = mrrmse(y_test,y_pred)
print(f' MRRSE Score: {mrrse_score:.3f}')

 MRRSE Score: 34.696


In [ ]:
feature_names = x_train.columns

In [ ]:
def ModelLinearRegression(x_train, y_train, x_test,y_test):
  scaler = StandardScaler()
  scaled_train = scaler.fit_transform(x_train)
  scaled_test = scaler.transform(x_test)
  LR = LinearRegression()
  LR.fit(scaled_train,y_train)
  y_pred = LR.predict(scaled_test)
  mrrse_score = mrrmse(y_test,y_pred)
  print(f' MRRSE Score: {mrrse_score:.3f}')
ModelLinearRegression(x_train, y_train, x_test,y_test)

 MRRSE Score: 25.514


In [ ]:
def DecisionTree(x_train, y_train,x_test,y_test):
  params = {
      "criterion":["squared_error", "absolute_error"],
      "max_depth":[2,4,8,16],
      "min_samples_split":[2],
      "min_samples_leaf":[2],
  }
  DT = DecisionTreeRegressor()
  g_cv = GridSearchCV(DT(random_state=1),
                          param_grid=params,scoring=mrrmse_scorer, cv=10,refit=True)
  g_cv.fit(x_train,y_train)
  best_params = g_cv.best_params_
  tree = DT(**best_params,random_state=1)
  tree.fit(x_train,y_train)
  y_pred = tree.predict(x_test)
  mrrse_score = mrrmse(y_test,y_pred)
  print(f' MRRSE Score: {mrrse_score:.3f}')

In [ ]:
LR = LinearRegression()
LR.fit(x_train,y_train)
y_pred = LR.predict(x_test)
mrrse_score = mrrmse(y_test,y_pred)
print(f' MRRSE Score: {mrrse_score}')

 MRRSE Score: 3.579464623126967e-15


In [ ]:
random_forest = RandomForestRegressor(random_state=12345)
random_forest.fit(x_train,y_train)
RF_Y_pred = random_forest.predict(x_test)
RF_mrrse_score = mrrmse(y_test,RF_Y_pred)
print(f' MRRSE Score: {RF_mrrse_score}')

 MRRSE Score: 0.35819877331089534


In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
DT = DecisionTreeRegressor()
DT.fit(x_train,y_train)
DT_y_pred = DT.predict(x_test)
DT_mrrse_score = mrrmse(y_test,DT_y_pred)
print(f' MRRSE Score: {DT_mrrse_score}')

 MRRSE Score: 0.7324833693455693


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [ ]:
KN = KNeighborsRegressor()
KN.fit(x_train,y_train)
KN_pred = KN.predict(x_test)
KN_mrrse_score = mrrmse(KN_pred,y_test)
print(f' MRRSE Score: {KN_mrrse_score}')

 MRRSE Score: 0.5024482331358064
